In [1]:
import folium
import json
import geopandas as gpd
import pandas as pd
from folium import GeoJsonTooltip
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np


#### First we get the Microsoft predictions for the entire US and we filter for North Carolina, Nebraska, Mississippi and Arkansas

In [2]:
gdf = gpd.read_file('../data/full-usa-3-13-2021_filtered_deduplicated.gpkg')
gdf_geojson = gdf.to_json()

In [10]:
# Nebraska
nebraska = gpd.read_file('../data/geojson_to_filter_out/state_boundary/nebraska.geojson')
ne_gdf = gpd.GeoDataFrame.from_features(nebraska)

polygon = ne_gdf.geometry.iloc[0] 
gdf['nebraska'] = gdf.geometry.within(polygon)
ne_predictions = gdf[gdf['nebraska']]

ne_predictions.to_file("../output/ne_predictions.geojson", driver='GeoJSON')

In [11]:
# North Carolina
northcarolina = gpd.read_file('../data/geojson_to_filter_out/state_boundary/north_carolina.geojson')
nc_gdf = gpd.GeoDataFrame.from_features(northcarolina)

polygon = nc_gdf.geometry.iloc[0] 
gdf['northcarolina'] = gdf.geometry.within(polygon)
nc_predictions = gdf[gdf['northcarolina']]

nc_predictions.to_file("../output/northcarolina_predictions.geojson", driver='GeoJSON')

In [3]:
# Arkansas
arkansas = gpd.read_file('../data/geojson_to_filter_out/state_boundary/arkansas.geojson')
ar_gdf = gpd.GeoDataFrame.from_features(arkansas)

polygon = ar_gdf.geometry.iloc[0] 
gdf['arkansas'] = gdf.geometry.within(polygon)
ar_predictions = gdf[gdf['arkansas']]

ar_predictions.to_file("../output/arkansas_predictions.geojson", driver='GeoJSON')

In [3]:
# Mississippi
mississippi = gpd.read_file('../data/geojson_to_filter_out/state_boundary/mississippi.geojson')
mi_gdf = gpd.GeoDataFrame.from_features(mississippi)

polygon = mi_gdf.geometry.iloc[0] 
gdf['mississippi'] = gdf.geometry.within(polygon)
mi_predictions = gdf[gdf['mississippi']]

mi_predictions.to_file("../output/mississippi_predictions.geojson", driver='GeoJSON')

#### Run the rule_base_filtering.py to get final data

#### Plot the final data that has been filtered by the rule_base_filtering.py with hovering labels, on click it will save the index and also add the marker to the map

In [3]:
from ipyleaflet import Map, GeoJSON, WidgetControl, TileLayer, Marker
from ipywidgets import HTML, Layout
import json
import threading
import time
import random

# Load your GeoJSON data
with open('../output/nc_predictions.geojson') as f:
    geojson_data = json.load(f)
for i, feature in enumerate(geojson_data['features']):
    # Assign an 'index' property to each feature
    feature['properties']['index'] = i
# Initialize the map
m = Map(center=(34.9, -92.3), zoom=6.5)
# Widget to display feature information on the map
widget = HTML()
widget.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=widget, position='bottomright')
m.add_control(control)
# Global variable to keep track of hover state
is_hovering = False
is_clicking = False
false_positives = []
# Define a hover handler
def handle_hover(event, feature, **kwargs):
    global is_hovering
    is_hovering = True
    properties = feature['properties']
    geometry = feature['geometry']['coordinates'][0][0]
    longitude, latitude = geometry[:2]
    index = feature['properties']['index']
    # Update the widget to show information
    widget.value = f'Index: {index}, Lat: {latitude}, Lon: {longitude}'
def handle_click(event, feature, **kwargs):
    global is_hovering, false_positives, m
    is_clicking = True
    index = feature['properties']['index']
    false_positives.append(index)
    print(f'Feature Index: {index}')
    longitude, latitude = feature['geometry']['coordinates'][0][0][:2]
    center = (latitude, longitude)
    marker = Marker(location=center, draggable=False)
    m.add(marker)


# Function to clear the widget if not hovering
def clear_widget(*args, **kwargs):
    global is_hovering
    if not is_hovering:
        widget.value = ''
    is_hovering = False

def change_click_color(feature):
    return {
        'color': 'blue',
        'fillColor': 'blue'
    }


# Create the GeoJSON layer with hover functionality and add it to the map
geo_json = GeoJSON(
    data=geojson_data,
    style={'color': 'yellow', 'fillcolor': 'yellow', 'opacity': 0.5},
    hover_style={'color': 'green', 'fillOpacity': 0.8},
    style_callback = change_click_color
)
geo_json.on_click(handle_click)
geo_json.on_hover(handle_hover)


esri_satellite = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attribution='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
m.add_layer(esri_satellite)


m.add_layer(geo_json)
m

Map(center=[34.9, -92.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

#### Create a validation set of 100 barns

In [4]:
from ipyleaflet import Map, GeoJSON, WidgetControl, TileLayer, Marker
from ipywidgets import HTML, Layout
import json
import threading
import time
import random

# Load your GeoJSON data
with open('../output/nc_predictions.geojson') as f:
    geojson_data = json.load(f)
for i, feature in enumerate(geojson_data['features']):
    # Assign an 'index' property to each feature
    feature['properties']['index'] = i

# Function to generate bbox that contains at least 100 barns
def generate_bbox_with_100_barns(features):
    attempts = 0
    while attempts < 1000:  # Limit attempts to avoid an infinite loop
        center_lat = random.uniform(33.0, 36.5)
        center_lon = random.uniform(-94.6, -89.6)
        size = 0.01  # Start with a small bbox and increase if needed
        while True:
            lat_min, lat_max = center_lat - size, center_lat + size
            lon_min, lon_max = center_lon - size, center_lon + size
            filtered_features = [
                feature for feature in features if
                lon_min <= feature['geometry']['coordinates'][0][0][0] <= lon_max and
                lat_min <= feature['geometry']['coordinates'][0][0][1] <= lat_max
            ]
            if len(filtered_features) >= 100:
                print(f"BBox coordinates: Lon ({lon_min}, {lon_max}), Lat ({lat_min}, {lat_max})")
                print(f"Total barns in this bbox: {len(filtered_features)}")  
                return filtered_features, (lat_min, lat_max, lon_min, lon_max)
            size += 0.01  # Increase bbox size
        attempts += 1
    raise ValueError("Failed to find a suitable bbox with 100 barns")

filtered_features, bbox_coords = generate_bbox_with_100_barns(geojson_data['features'])

# Initialize the map at the center of the bbox
center_lat = (bbox_coords[0] + bbox_coords[1]) / 2
center_lon = (bbox_coords[2] + bbox_coords[3]) / 2
m = Map(center=(center_lat, center_lon), zoom=7)

# Create the GeoJSON layer with hover functionality and add it to the map
geo_json = GeoJSON(
    data={'type': 'FeatureCollection', 'features': filtered_features},
    style={'color': 'red', 'fillcolor': '#ffffff', 'opacity': 1.0, 'fillOpacity': 0},
    hover_style={'color': 'green', 'fillOpacity': 0.8},
    style_callback=change_click_color
)
geo_json.on_click(handle_click)
geo_json.on_hover(handle_hover)

# Add other map components as before
widget = HTML()
widget.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=widget, position='bottomright')
m.add_control(control)
esri_satellite = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attribution='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
m.add_layer(esri_satellite)
m.add_layer(geo_json)
m


BBox coordinates: Lon (-102.15362648509526, -83.87362648509557), Lat (25.132474291349958, 43.41247429134966)
Total barns in this bbox: 126


Map(center=[34.27247429134981, -93.01362648509541], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Feature Index: 20695
Feature Index: 20696
Feature Index: 20699
Feature Index: 20700


In [10]:
with open('../output/index.txt', 'a') as file:
    for i in false_positives:
        file.write(str(i)+',')